# Load packages

In [1]:
!pip uninstall -y kfp

Found existing installation: kfp 1.0.4
Uninstalling kfp-1.0.4:
  Successfully uninstalled kfp-1.0.4


In [19]:
!pip install kfp

In [3]:
import kfp
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

/opt/conda/lib/python3.8/site-packages/kubernetes/utils/create_from_yaml.py:77: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if kind is not "":


In [4]:
kfp.__version__

'1.4.0'

# Enter your gateway loadbalancer and the token from the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

In [3]:
#Update values for the load balancer and auth session
HOST='istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="authservice_session=MTYxOTYzMTk4MnxOd3dBTkUxVlVWVlpXbGxPTkRaQk5WcEpXa3REVEZJMVNUSlJVRmRQVjBkTVYwRkdRMWhRUVZGUk0xb3pSRFJTU0RkYU5rVmFOMEU9fDId_ifCUhyqdfHgx_vRUuBQYIZu8omTjr1w6US0vNJT"
namespace="kubeflow-user-example-com"

In [6]:
client = kfp.Client(host=HOST+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace=namespace)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

### Using dsl for model archiver and torchserve integration
#### Init container is used for mar file generation

In [7]:
DEPLOY="torchserve"
MODEL="cifar10"

In [8]:
mar_op = load_component_from_file("./model_archive/component.yaml")
deploy_op = load_component_from_file("./deploy/component.yaml")

In [9]:
prep_op = components.load_component_from_file(
    "./pytorch/data_prep/component.yaml"
)
train_op = components.load_component_from_file(
    "./pytorch/train/component.yaml"
)

In [10]:
@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_bert():

    @dsl.component
    def ls(input_dir: str):
        return dsl.ContainerOp(
            name='list',
            image='busybox:latest',
            command=["ls", "-R", "%s" % input_dir]
        )
    
    prep_task=prep_op()
    train_task=train_op(prep_task.outputs['output_data']).after(prep_task)
    
    
    deploy_task = deploy_op(
        action="apply",
        model_name="%s" % DEPLOY,
        model_uri="s3://mlpipeline/mar",
        namespace="%s" % namespace,
        framework='pytorch',
        service_account='sa'
    ).after(train_task)
    

In [11]:
# Compile pipeline
compiler.Compiler().compile(pytorch_bert, 'pytorch.tar.gz', type_check=True)

In [12]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-cifar10', 'pytorch.tar.gz')

### Wait for inference service below to go to `READY True` state.

In [13]:
!kubectl get isvc $DEPLOY

NAME         URL                                                       READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                  AGE
torchserve   http://torchserve.kubeflow-user-example-com.example.com   True           100                              torchserve-predictor-default-zwl6p   114s


# Inference

In [256]:
!kubectl get isvc $DEPLOY -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3

torchserve-bert.admin.example.com


In [257]:
S_HOSTNAME=!kubectl get isvc $DEPLOY  -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3
SERVICE_HOSTNAME=S_HOSTNAME[0]
SERVICE_HOSTNAME

'torchserve-bert.admin.example.com'

# Prediction

In [259]:
!cat sample.txt

{
  "instances": [
    {
      "data": "Bloomberg has reported on the economy"
    }
  ]
}
